# Sequence Analysis with Python

Contact: Veli Mäkinen veli.makinen@helsinki.fi

The following assignments introduce applications of hashing with ```dict()``` primitive of Python. While doing so, a rudimentary introduction to biological sequences is given. 
This framework is then enhanced with probabilities, leading to routines to generate random sequences under some constraints, including a general concept of *Markov-chains*. All these components illustrate the usage of ```dict()```, but at the same time introduce some other computational routines to efficiently deal with probabilities.   
The function ```collections.defaultdict``` can be useful.

Below are some "suggested" imports. Feel free to use and modify these, or not. Generally it's good practice to keep most or all imports in one place. Typically very close to the start of notebooks.

In [54]:
from collections import defaultdict, Counter
from itertools import product
from typing import Any, Dict, Iterator, List # added for type hints

import numpy as np
from numpy import random
from numpy.random import choice
import pandas as pd

The automated TMC tests do not test cell outputs. These are intended to be evaluated in the peer reviews. So it is still be a good idea to make the outputs as clear and informative as possible.

To keep TMC tests running as well as possible it is recommended to keep global variable assignments in the notebook to a minimum to avoid potential name clashes and confusion. Additionally you should keep all actual code exection in main guards to keep the test running smoothly. If you run [check_sequence.py](https://raw.githubusercontent.com/saskeli/data-analysis-with-python-summer-2019/master/check_outputs.py) in the `part07-e01_sequence_analysis` folder, the script should finish very quickly and optimally produce no output.

If you download data from the internet during execution (codon usage table), the parts where downloading is done should not work if you decide to submit to the tmc server. Local tests should work fine.

## DNA and RNA

A DNA molecule consist, in principle, of a chain of smaller molecules. These smaller molecules have some common basic components (bases) that repeat. For our purposes it is sufficient to know that these bases are nucleotides adenine, cytosine, guanine, and thymine with abbreviations ```A```, ```C```, ```G```, and ```T```. Given a *DNA sequence* e.g. ```ACGATGAGGCTCAT```, one can reverse engineer (with negligible loss of information) the corresponding DNA molecule.

Parts of a DNA molecule can *transcribe* into an RNA molecule. In this process, thymine gets replaced by uracil (```U```). 


1. Write a function ```dna_to_rna``` to convert a given DNA sequence $s$ into an RNA sequence. For the sake of exercise, use ```dict()``` to store the symbol to symbol encoding rules. Create a program to test your function.

In [55]:

def dna_to_rna(dna: str) -> str:
    transcription = dict(zip("ACGT", "ACGU"))

    return "".join(map(transcription.get, dna))
    
if __name__ == '__main__':
    print(dna_to_rna("AACGTGCTTTC"))

AACGUGCUUUC


### Idea of solution

The mapping is produced by zipping two strings containing the DNA and RNA equivalents in their respective order and converting the result into a dict.

This dict is then used in mapping the argument DNA string into RNA.

### Discussion

The input is expected to be a valid DNA string, as there are no checks for invalid inputs.

## Proteins

Like DNA and RNA, protein molecule can be interpreted as a chain of smaller molecules, where the bases are now amino acids. RNA molecule may *translate* into a protein molecule, but instead of base by base, three bases of RNA correspond to one base of protein. That is, RNA sequence is read triplet (called codon) at a time. 

2. Consider the codon to amino acid conversion table in http://htmlpreview.github.io/?https://github.com/csmastersUH/data_analysis_with_python_2020/blob/master/Codon%20usage%20table.html. Write a function ```get_dict``` to read the table into a ```dict()```, such that for each RNA sequence of length 3, say $\texttt{AGU}$, the hash table stores the conversion rule to the corresponding amino acid. You may store the html page to your local src directory,
and parse that file.

In [56]:

def read_codon_table() -> pd.DataFrame:
    """
    Helper function for loading the Codon usage table that works both for running the notebook and while running the tests (requires the "src/" prefix).

    Defined separately here as adding/removing the said prefix repeatedly in later exercises became cumbersome. Now it can be called by future functions.
    """
    try:
        # For running
        df = pd.read_csv("Codon_usage_table.csv", sep=";")
    except FileNotFoundError:
        # For tests
        df = pd.read_csv("src/Codon_usage_table.csv", sep=";")
    
    df = df.dropna(how="all").dropna(axis=1, how="all")
    return df

def get_dict() -> Dict[str, str]:
    """
    Returns a dictionary that maps RNA triplets (codons) to amino acids (proteins) they convert to.
    """
    data = read_codon_table()
    return dict(zip(data["Triplet"], data["Amino acid"]))
    
if __name__ == '__main__':
    codon_to_aa = get_dict()
    print(codon_to_aa)

{'UUU': 'F', 'UCU': 'S', 'UAU': 'Y', 'UGU': 'C', 'UUC': 'F', 'UCC': 'S', 'UAC': 'Y', 'UGC': 'C', 'UUA': 'L', 'UCA': 'S', 'UAA': '*', 'UGA': '*', 'UUG': 'L', 'UCG': 'S', 'UAG': '*', 'UGG': 'W', 'CUU': 'L', 'CCU': 'P', 'CAU': 'H', 'CGU': 'R', 'CUC': 'L', 'CCC': 'P', 'CAC': 'H', 'CGC': 'R', 'CUA': 'L', 'CCA': 'P', 'CAA': 'Q', 'CGA': 'R', 'CUG': 'L', 'CCG': 'P', 'CAG': 'Q', 'CGG': 'R', 'AUU': 'I', 'ACU': 'T', 'AAU': 'N', 'AGU': 'S', 'AUC': 'I', 'ACC': 'T', 'AAC': 'N', 'AGC': 'S', 'AUA': 'I', 'ACA': 'T', 'AAA': 'K', 'AGA': 'R', 'AUG': 'M', 'ACG': 'T', 'AAG': 'K', 'AGG': 'R', 'GUU': 'V', 'GCU': 'A', 'GAU': 'D', 'GGU': 'G', 'GUC': 'V', 'GCC': 'A', 'GAC': 'D', 'GGC': 'G', 'GUA': 'V', 'GCA': 'A', 'GAA': 'E', 'GGA': 'G', 'GUG': 'V', 'GCG': 'A', 'GAG': 'E', 'GGG': 'G'}


### Idea of solution

The table, extracted from the HTML page, is formatted into a semicolon-separated table in Codon_usage_table.csv under the local src directory. I use pandas to read this table, and then use the "Triplet" and "Amino acid" -columns to form a dict-object by zipping them together into a mapping.

### Discussion

The following changes were made to the column names in the codon usage table to make their use easier: "frequency: per thousand" -> "Frequency" and "([number])" -> "Number. Additionally, the other column names were given capital initials.

3. Use the same conversion table as above, but now write function `get_dict_list` to read the table into a `dict()`, such that for each amino acid the hash table stores the list of codons encoding it.    

In [57]:

def get_dict_list() -> Dict[str, List[str]]:
    """
    Returns a dictionary that maps amino acids (proteins) to a list of RNA triplets (codons) that they can be converted from.
    """
    aa_to_codons = defaultdict(list)
    
    for codon, aa in get_dict().items():
        aa_to_codons[aa].append(codon)
    
    return aa_to_codons
    
if __name__ == '__main__':
    aa_to_codons = get_dict_list()
    print(aa_to_codons)

defaultdict(<class 'list'>, {'F': ['UUU', 'UUC'], 'S': ['UCU', 'UCC', 'UCA', 'UCG', 'AGU', 'AGC'], 'Y': ['UAU', 'UAC'], 'C': ['UGU', 'UGC'], 'L': ['UUA', 'UUG', 'CUU', 'CUC', 'CUA', 'CUG'], '*': ['UAA', 'UGA', 'UAG'], 'W': ['UGG'], 'P': ['CCU', 'CCC', 'CCA', 'CCG'], 'H': ['CAU', 'CAC'], 'R': ['CGU', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'], 'Q': ['CAA', 'CAG'], 'I': ['AUU', 'AUC', 'AUA'], 'T': ['ACU', 'ACC', 'ACA', 'ACG'], 'N': ['AAU', 'AAC'], 'K': ['AAA', 'AAG'], 'M': ['AUG'], 'V': ['GUU', 'GUC', 'GUA', 'GUG'], 'A': ['GCU', 'GCC', 'GCA', 'GCG'], 'D': ['GAU', 'GAC'], 'G': ['GGU', 'GGC', 'GGA', 'GGG'], 'E': ['GAA', 'GAG']})


### Idea of solution

The function calls the previously defined get_dict function and inverses the mapping. The original map's values are used as keys, and duplicate keys are combined into a list and stored as the inverse map's values.

### Discussion

I'm not convinced the approach I chose is the fastest method for inversing the mapping but it was the best I could think of.

With the conversion tables at hand, the following should be trivial to solve.

4. Fill in function ```rna_to_prot``` in the stub solution to convert a given DNA sequence $s$ into a protein sequence. 
You may use the dictionaries from exercises 2 and 3. You can test your program with `ATGATATCATCGACGATGTAG`.

In [58]:

def rna_to_prot(rna: str) -> str:
    # Generate codons of length 3 from the input RNA string.
    codons = (rna[i:i+3] for i in range(0, len(rna), 3))

    return "".join(map(get_dict().get, codons))

def dna_to_prot(dna_seq: str) -> str:
    return rna_to_prot(dna_to_rna(dna_seq))

if __name__ == '__main__':
    print(dna_to_prot("ATGATATCATCGACGATGTAG"))

MISSTM*


### Idea of solution

The RNA string is split into codons, each of which are then converted into a protein sequence using the previously defined get_dict function.

### Discussion

I initially used a List comprehension to produce the codons. To reduce the memory cost at runtime, I decided to change it into a Generator. The input string is assumed to be valid as there are no checks for e.g. Nones or invalid characters.

You may notice that there are $4^3=64$ different codons, but only 20 amino acids. That is, some triplets encode the same amino acid.  

## Reverse translation

It has been observed that among the codons coding the same amino acid, some are more frequent than others. These frequencies can be converted to probabilities. E.g. consider codons `AUU`, `AUC`, and `AUA` that code for amino acid isoleucine.
If they are observed, say, 36, 47, 17 times, respectively, to code isoleucine in a dataset, the probability that a random such event is `AUU` $\to$ isoleucine is 36/100.

This phenomenon is called *codon adaptation*, and for our purposes it works as a good introduction to generation of random sequences under constraints.   

5. Consider the codon adaptation frequencies in http://htmlpreview.github.io/?https://github.com/csmastersUH/data_analysis_with_python_2020/blob/master/Codon%20usage%20table.html and read them into a ```dict()```, such that for each RNA sequence of length 3, say `AGU`, the hash table stores the probability of that codon among codons encoding the same amino acid.
Put your solution in the ```get_probabability_dict``` function. Use the column "([number])" to estimate the probabilities, as the two preceding columns contain truncated values.  

In [59]:

def get_probability_dict() -> Dict[str, float]:
    data = read_codon_table()
    data["Probability"] = data.groupby("Amino acid")["Number"].transform(lambda x: x / x.sum())

    probability_dict = defaultdict(float, zip(data["Triplet"], data["Probability"]))
    
    return probability_dict

def get_probabability_dict() -> Dict[str, float]:
    """
    Returns a dictionary that maps codons to their the probabability [sic] of their occurrence among codons that convert into the same protein.
    """
    return get_probability_dict()
    
if __name__ == '__main__':
    codon_to_prob = get_probabability_dict()
    items = sorted(codon_to_prob.items(), key=lambda x: x[0])
    for i in range(1 + len(items)//6):
        print("\t".join(
            f"{k}: {v:.6f}"
            for k, v in items[i*6:6+i*6]
        ))

AAA: 0.434049	AAC: 0.529633	AAG: 0.565951	AAU: 0.470367	ACA: 0.284188	ACC: 0.355232
ACG: 0.113812	ACU: 0.246769	AGA: 0.214658	AGC: 0.239938	AGG: 0.211091	AGU: 0.149602
AUA: 0.169062	AUC: 0.469866	AUG: 1.000000	AUU: 0.361072	CAA: 0.265017	CAC: 0.581485
CAG: 0.734983	CAU: 0.418515	CCA: 0.276603	CCC: 0.323470	CCG: 0.113196	CCU: 0.286731
CGA: 0.108812	CGC: 0.183777	CGG: 0.201554	CGU: 0.080108	CUA: 0.071380	CUC: 0.195577
CUG: 0.395702	CUU: 0.131716	GAA: 0.422453	GAC: 0.535458	GAG: 0.577547	GAU: 0.464542
GCA: 0.228121	GCC: 0.399781	GCG: 0.106176	GCU: 0.265922	GGA: 0.249922	GGC: 0.337109
GGG: 0.249882	GGU: 0.163087	GUA: 0.116577	GUC: 0.238306	GUG: 0.463346	GUU: 0.181770
UAA: 0.297019	UAC: 0.556662	UAG: 0.236738	UAU: 0.443338	UCA: 0.150517	UCC: 0.217960
UCG: 0.054398	UCU: 0.187586	UGA: 0.466243	UGC: 0.543843	UGG: 1.000000	UGU: 0.456157
UUA: 0.076568	UUC: 0.535866	UUG: 0.129058	UUU: 0.464134


### Idea of solution

The function loads the data from the same file as the earlier exercise. It then groups it by the "Amino acid" column to calculate a probability for each codon, which is stored in a new column called "Probability". This column is then used, together with the "Triplet" column, to produce a mapping from RNA sequences to their in-group probabilities.

### Discussion

A defaultdict was used as a dictionary of choice because it is able to return a default probability of 0.0 if a codon that it does not contain a value for is given as input.

Now you should have everything in place to easily solve the following.


6. Write a class ```ProteinToMaxRNA``` with a ```convert``` method which converts a protein sequence into the most likely RNA sequence to be the source of this protein. Run your program with `LTPIQNRA`.

In [60]:
class ProteinToMaxRNA:
    
    def __init__(self):
        self.protein_to_codon = get_dict_list()
        self.rna_probabilities = get_probabability_dict()

    def convert(self, protein: str) -> str:
        """
        Converts a protein base by base into the likeliest codons it could consist of.

        Args:
            protein (str): A string consisting of valid protein bases.
        
        Returns:
            str: A string consisting of the likeliest codons that convert to the provided protein.
        """
        codons = []

        for base in protein:
            codons_ = self.protein_to_codon[base]
            probabilities = list(map(self.rna_probabilities.get, codons_))

            codons.append(codons_[np.argmax(probabilities)])
        
        return "".join(codons)


if __name__ == '__main__':
    protein_to_rna = ProteinToMaxRNA()
    print(protein_to_rna.convert("LTPIQNRA"))

CUGACCCCCAUCCAGAACAGAGCC


### Idea of solution

The constructor of the class stores the maps for protein-to-RNA and RNA-to-probabilities and uses them to pick the RNA with the highest probability, using numpy's argmax, and adds that RNA to a list. This list is then combined to a string to produce the resulting RNA sequence consisting of the likeliest codons.

### Discussion

The solution is a simple combination of previously defined functions. Like before, the input is expected to be correct as there are no checks for Nones or invalid strings.

Now we are almost ready to produce random RNA sequences that code a given protein sequence. For this, we need a subroutine to *sample from a probability distribution*. Consider our earlier example of probabilities 36/100, 47/100, and 17/100 for `AUU`, `AUC`, and `AUA`, respectively. 
Let us assume we have a random number generator ```random()``` that returns a random number from interval $[0,1)$. We may then partition the unit interval according to cumulative probabilities to $[0,36/100), [36/100,83/100), [83/100,1)$, respectively. Depending which interval the number ```random()``` hits, we select the codon accordingly.

7. Write a function ```random_event``` that chooses a random event, given a probability distribution (set of events whose probabilities sum to 1).
You can use function ```random.uniform``` to produce values uniformly at random from the range $[0,1)$. The distribution should be given to your function as a dictionary from events to their probabilities.

In [61]:
def random_event(dist: Dict[str, float], seed = None) -> str:
    """
    Takes as input a dictionary from events to their probabilities.
    Return a random event sampled according to the given distribution.
    The probabilities must sum to 1.0
    """
    rng = random.RandomState(seed)
    
    upper_limits = np.cumsum(list(dist.values()))

    xi = rng.uniform()
    choice_idx = np.sum(upper_limits < xi)

    return list(dist.keys())[choice_idx]

if __name__ == '__main__':
    distribution = dict(zip("ACGT", [0.10, 0.35, 0.15, 0.40]))
    print(", ".join(random_event(distribution) for _ in range(29)))

T, T, C, G, G, C, C, G, G, C, G, C, T, T, T, A, C, C, C, T, T, G, A, C, T, G, G, C, T


### Idea of solution

The function samples a random variable, $\xi$ (xi), from the uniform distribution [0, 1]. It then calculates the cumulative sums with the values of the provided distribution, forming the upper limit for each respective key. These cumulative sums are then compared to the sampled random variable to find out the number of keys whose upper limit is below $\xi$. This number is the index of the key, in the distribution, that corresponds to the sampled random variable (e.g. if xi is below the n'th key's limit, then the count is $n-1$, which is the index of the $n$'th key).

### Discussion

The solution is a simple selection using a random variable and a list of cumulative sums. The received input is not checked for correctness.

With this general routine, the following should be easy to solve.
 
8. Write a class ```ProteinToRandomRNA``` to produce a random RNA sequence encoding the input protein sequence according to the input codon adaptation probabilities. The actual conversion is done through the ```convert``` method. Run your program with `LTPIQNRA`.

In [62]:
class ProteinToRandomRNA(ProteinToMaxRNA):

    def convert(self, protein: str) -> str:
        """
        Converts a protein base by base into randomly selected codons it could consist of.

        Args:
            protein (str): A string consisting of valid protein bases.
        
        Returns:
            str: A string consisting of randomly selected codons that convert to the provided protein.
        """
        codons = []

        for base in protein:
            codons_ = self.protein_to_codon[base]
            probabilities = list(map(self.rna_probabilities.get, codons_))
            distribution = dict(zip(codons_, probabilities))

            codons.append(random_event(distribution))
        
        return "".join(codons)
        
if __name__ == '__main__':
    protein_to_random_codons = ProteinToRandomRNA()
    print(protein_to_random_codons.convert("LTPIQNRA"))

CUUACUCCCAUACAGAACAGGGCC


### Idea of solution

The primary difference to ProteinToMaxRNA is that instead of choosing the likeliest codon, this class chooses a random sample with the previously defined random_event function.

### Discussion

The input is not checked for correctness.

## Generating DNA sequences with higher-order Markov chains

We will now reuse the machinery derived above in a related context. We go back to DNA sequences, and consider some easy statistics that can be used to characterize the sequences. 
First, just the frequencies of bases $\texttt{A}$, $\texttt{C}$, $\texttt{G}$, $\texttt{T}$ may reveal the species from which the input DNA originates; each species has a different base composition that has been formed during evolution. 
More interestingly, the areas where DNA to RNA transcription takes place (coding region) have an excess of $\texttt{C}$ and $\texttt{G}$ over $\texttt{A}$ and $\texttt{T}$. To detect such areas a common routine is to just use a *sliding window* of fixed size, say $k$, and compute for each window position 
$T[i..i+k-1]$ the base frequencies, where $T[1..n]$ is the input DNA sequence. When sliding the window from  $T[i..i+k-1]$ to $T[i+1..i+k]$ frequency $f(T[i])$ gets decreases by one and $f(T[i+k])$ gets increased by one. 

9. Write a *generator* ```sliding_window``` to compute sliding window base frequencies so that each moving of the window takes constant time. We saw in the beginning of the course one way how to create generators using
  generator expression. Here we use a different way. For the function ```sliding_window``` to be a generator, it must have at least   one ```yield``` expression, see [https://docs.python.org/3/reference/expressions.html#yieldexpr](https://docs.python.org/3/reference/expressions.html#yieldexpr).
  
  Here is an example of a generator expression that works similarily to the built in `range` generator:
  ```Python
  def range(a, b=None, c=1):
      current = 0 if b == None else a
      end = a if b == None else b
      while current < end:
          yield current
          current += c
  ```
  A yield expression can be used to return a value and *temporarily* return from the function.

In [63]:
def sliding_window(s: str, k: int) -> Iterator[Dict[str, int]]:
    """
    This function returns a generator that can be iterated over all
    starting position of a k-window in the sequence.
    For each starting position the generator returns the nucleotide frequencies
    in the window as a dictionary.
    """
    zeros = dict(zip("ACGT", [0]*4))
    for i in range(len(s)-k+1):
        counts = Counter(s[i:i+k])
        counts.update(zeros) # add missing nucleotides
        yield dict(counts)
        
    
if __name__ == '__main__':
    s = "TCCCGACGGCCTTGCC"
    for d in sliding_window(s, 4):
        print(d)

{'T': 1, 'C': 3, 'A': 0, 'G': 0}
{'C': 3, 'G': 1, 'A': 0, 'T': 0}
{'C': 2, 'G': 1, 'A': 1, 'T': 0}
{'C': 2, 'G': 1, 'A': 1, 'T': 0}
{'G': 2, 'A': 1, 'C': 1, 'T': 0}
{'A': 1, 'C': 1, 'G': 2, 'T': 0}
{'C': 2, 'G': 2, 'A': 0, 'T': 0}
{'G': 2, 'C': 2, 'A': 0, 'T': 0}
{'G': 1, 'C': 2, 'T': 1, 'A': 0}
{'C': 2, 'T': 2, 'A': 0, 'G': 0}
{'C': 1, 'T': 2, 'G': 1, 'A': 0}
{'T': 2, 'G': 1, 'C': 1, 'A': 0}
{'T': 1, 'G': 1, 'C': 2, 'A': 0}


### Idea of solution

Counter is used to calculate the number of occurrences of each nucleotide in each $k$-window, and then updated with a Counter instance containing zeroes for all four nucleotides to ensure they are all present in the resulting dictionary.

### Discussion

I initially implemented this as a while-loop but then realised it could be simplified and made easier to follow by turning it into a for-loop.

 
Our models so far have been so-called *zero-order* models, as each event has been independent of other events. With sequences, the dependencies of events are naturally encoded by their *contexts*. Considering that a sequence is produced from left-to-right, a *first-order* context for $T[i]$ is $T[i-1]$, that is, the immediately preceding symbol. *First-order Markov chain* is a sequence produced by generating $c=T[i]$ with the probability of event of seeing symbol $c$ after previously generated symbol $a=T[i-1]$. The first symbol of the chain is sampled according to the zero-order model.  
The first-order model can naturally be extended to contexts of length $k$, with $T[i]$ depending on $T[i-k..i-1]$. Then the first $k$ symbols of the chain are sampled according to the zero-order model.  The following assignments develop the routines to work with the *higher-order Markov chains*. 
In what follows, a $k$-mer is a substring $T[i..i+k-1]$ of the sequence at an arbitrary position. 

10. Write function ```context_list``` that given an input DNA sequence $T$ associates to each $k$-mer $W$ the concatenation of all symbols $c$ that appear after context $W$ in $T$, that is, $T[i..i+k]=Wc$. For example, <span style="color:red; font:courier;">GA</span> is associated to <span style="color:blue; font: courier;">TCT</span> in $T$=<span style="font: courier;">AT<span style="color:red;">GA</span><span style="color:blue;">T</span>ATCATC<span style="color:red;">GA</span><span style="color:blue;">C</span><span style="color:red;">GA</span><span style="color:blue;">T</span>GTAG</span>, when $k=2$.

In [64]:
def context_list(seq: str, k: int, pseudo_fill: str = None) -> Dict[str, str]:
    contexts = defaultdict(str)
    
    if pseudo_fill is not None:
        for context in ("".join(p) for p in product("ACGT", repeat=k)):
            contexts[context] = pseudo_fill

    for i in range(len(seq) - k):
        context = seq[i:i+k]
        symbol = seq[i+k]

        contexts[context] += symbol

    return contexts
    
if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATCTAG"
    d = context_list(s, k)
    print(d)

defaultdict(<class 'str'>, {'AT': 'GACCC', 'TG': 'A', 'GA': 'TCT', 'TA': 'TG', 'TC': 'AGT', 'CA': 'T', 'CG': 'AA', 'AC': 'G', 'CT': 'A'})


### Idea of solution

The function scans the input string in $k+1$ -sized chunks, where the first $k$ characters form the context, and the last is the symbol associated with the context.

Additionally, I added the ability to provide a "pseudo-fill" string to enter a specific string to all of the keys in the dictionary, simplifying future exercises.

### Discussion

The solution was simplified through the use of defaultdict as each associated symbol could be simply appended to the default, empty, string for each new key.

11. With the above solution, write function ```context_probabilities``` to count the frequencies of symbols in each context and convert these frequencies into probabilities. Run `context_probabilities` with $T=$ `ATGATATCATCGACGATGTAG` and $k$ values 0 and 2.

In [65]:
def context_probabilities(seq: str, k: int) -> Dict[str, Dict[str, float]]:
    contexts = context_list(seq, k)
    probabilities = defaultdict(dict)

    zeros = dict(zip("ACGT", [0]*4))
    for context, symbols in contexts.items():
        counts = Counter(symbols)
        counts.update(zeros) # Add zeroes for missing symbols

        total = len(symbols)

        probabilities[context] = {sym: cnt / total for sym, cnt in counts.items()}

    return probabilities
    
if __name__ == '__main__':
    T = "ATGATATCATCGACGATGTAG"
    for k in 0, 1:
        print(context_probabilities(T, k))


defaultdict(<class 'dict'>, {'': {'A': 0.3333333333333333, 'T': 0.2857142857142857, 'G': 0.23809523809523808, 'C': 0.14285714285714285}})
defaultdict(<class 'dict'>, {'A': {'T': 0.7142857142857143, 'C': 0.14285714285714285, 'G': 0.14285714285714285, 'A': 0.0}, 'T': {'G': 0.3333333333333333, 'A': 0.3333333333333333, 'C': 0.3333333333333333, 'T': 0.0}, 'G': {'A': 0.75, 'T': 0.25, 'C': 0.0, 'G': 0.0}, 'C': {'A': 0.3333333333333333, 'G': 0.6666666666666666, 'C': 0.0, 'T': 0.0}})


### Idea of solution

The function gets the context list from the previous exercise and then iterates through the contexts to calculate the probabilities for each potential symbol. Missing symbols are added at a probability of zero.

### Discussion

The number of for-loops, including the nested for loops in this one, feel unoptimal but I couldn't figure out a good way to avoid them.

12. With the above solution and the function ```random_event``` from the earlier exercise, write class ```MarkovChain```. Its ```generate``` method should generate a random DNA sequence following the original $k$-th order Markov chain probabilities. 

In [66]:
class MarkovChain:
    
    def __init__(self, zeroth: Dict[str, float], kth: Dict[str, Dict[str, float]], k: int=2):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
        
    def generate(self, n: int, seed=None) -> str:
        seq = []

        for i in range(n):
            if self.k == 0 or i < self.k:
                seq.append(random_event(self.zeroth, seed=seed))
                continue

            k_latest = "".join(seq[-self.k:])
            seq.append(random_event(self.kth[k_latest], seed=seed))

        return "".join(seq)

if __name__ == '__main__':
    zeroth = {'A': 0.2, 'C': 0.19, 'T': 0.31, 'G': 0.3}
    kth = {'GT': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0},
           'CA': {'A': 0.0, 'C': 0.0, 'T': 1.0, 'G': 0.0},
           'TC': {'A': 0.5, 'C': 0.0, 'T': 0.0, 'G': 0.5},
           'GA': {'A': 0.0, 'C': 0.3333333333333333, 'T': 0.6666666666666666, 'G': 0.0},
           'TG': {'A': 0.5, 'C': 0.0, 'T': 0.5, 'G': 0.0},
           'AT': {'A': 0.2, 'C': 0.4, 'T': 0.0, 'G': 0.4},
           'TA': {'A': 0.0, 'C': 0.0, 'T': 0.5, 'G': 0.5},
           'AC': {'A': 0.0, 'C': 0.0, 'T': 0.0, 'G': 1.0},
           'CG': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0}}
    n = 10
    seed = 0
    mc = MarkovChain(zeroth, kth)
    print(mc.generate(n, seed))

KeyError: 'TT'

### Idea of solution

The first $k$ members of the sequence are selected using the zeroth-distribution, and the remaining $n-k$ members are determined by combining the $k$ previous members as a key for the kth-distribution, which is then used to select a new member.

### Discussion

The provided seed is used with the numpy RandomState to provide deterministic results for testing purposes.

If you have survived so far without problems, please run your program a few more times with different inputs. At some point you should get a lookup error in your hash-table! The reason for this is not your code, but the way we defined the model: Some $k$-mers may not be among the training data (input sequence $T$), but such can be generated as the first $k$-mer that is generated using the zero-order model.  

A general approach to fixing such issues with incomplete training data is to use *pseudo counts*. That is, all imaginable events are initialized to frequency count 1.   

13. Write a new solution `context_pseudo_probabilities` based on the solution to problem 11. But this time use pseudo counts in order to obtain a $k$-th order Markov chain that can assign a probability for any DNA sequence. You may use the standard library function `itertools.product` to iterate over all $k$-mer of given length (`product("ACGT", repeat=k)`).

In [ ]:
def context_pseudo_probabilities(seq: str, k: int) -> Dict[str, Dict[str, float]]:
    contexts = context_list(seq, k, pseudo_fill="ACGT")
    probabilities = defaultdict(dict)

    for context, symbols in contexts.items():
        counts = Counter(symbols)
        total = len(symbols)

        probabilities[context] = {sym: cnt / total for sym, cnt in counts.items()}
    
    return dict(probabilities)
    
if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    kth = context_pseudo_probabilities(s, k)
    zeroth = context_pseudo_probabilities(s, 0)[""]
    print(f"zeroth: {zeroth}")
    print("\n".join(f"{k}: {dict(v)}" for k, v in kth.items()))
    print("\n", MarkovChain(zeroth, kth, k).generate(20))

zeroth: {'A': 0.32, 'C': 0.16, 'G': 0.24, 'T': 0.28}
AA: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AC: {'A': 0.2, 'C': 0.2, 'G': 0.4, 'T': 0.2}
AG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AT: {'A': 0.2222222222222222, 'C': 0.3333333333333333, 'G': 0.3333333333333333, 'T': 0.1111111111111111}
CA: {'A': 0.2, 'C': 0.2, 'G': 0.2, 'T': 0.4}
CC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
CG: {'A': 0.5, 'C': 0.16666666666666666, 'G': 0.16666666666666666, 'T': 0.16666666666666666}
CT: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GA: {'A': 0.14285714285714285, 'C': 0.2857142857142857, 'G': 0.14285714285714285, 'T': 0.42857142857142855}
GC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GT: {'A': 0.4, 'C': 0.2, 'G': 0.2, 'T': 0.2}
TA: {'A': 0.16666666666666666, 'C': 0.16666666666666666, 'G': 0.3333333333333333, 'T': 0.3333333333333333}
TC: {'A': 0.3333333333333333, 'C': 0.16666666666666666, 'G': 0.3333333333333333, 'T': 0.16666666666666666

### Idea of solution

The only difference to context_probabilities is that the default value for the counts is 1 instead of 0. This is achieved by passing the "ACGT" as a pseudo_fill parameter to the context_list function. Otherwise the solution is identical.

### Discussion

The nested for-loops are still something I take issue with but can't avoid.

14. Write class ```MarkovProb``` that given the $k$-th order Markov chain developed above to the constructor, its method ```probability``` computes the probability of a given input DNA sequence.

In [ ]:
class MarkovProb:
    def __init__(self, k: int, zeroth: Dict[str, float], kth: Dict[str, Dict[str, float]]):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
        
    def probability(self, seq: str) -> float:
        if not seq:
            return np.nan
        
        probability = 1.0
        for i, s in enumerate(seq):
            if i < self.k:
                probability *= self.zeroth[s]
                continue
            
            context = seq[i-self.k:i]
            probability *= self.kth[context][s]
        
        return probability

    
if __name__ == '__main__':
    k = 2
    kth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", k)
    zeroth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", 0)[""]
    mc = MarkovProb(2, zeroth, kth)
    s="ATGATATCATCGACGATGTAG"
    print(f"Probability of sequence {s} is {mc.probability(s)}")

Probability of sequence ATGATATCATCGACGATGTAG is 2.831270190340017e-10


### Idea of solution

The probability is calculated by multiplying the respective probabilities of each consecutive context, received from the distributions given during initialisation, together to form the result.

### Discussion

Repeated floating point multiplications can cause errors if the strings are long. Input is not validated.

With the last assignment you might end up in trouble with precision, as multiplying many small probabilities gives a really small number in the end. There is an easy fix by using so-called log-transform. 
Consider computation of $P=s_1 s_2 \cdots s_n$, where $0\leq s_i\leq 1$ for each $i$. Taking logarithm in base 2 from both sides gives $\log _2 P= \log _2 (s_1 s_2 \cdots s_n)=\log_2 s_1 + \log_2 s_2 + \cdots \log s_n= \sum_{i=1}^n \log s_i$, with repeated application of the property that the logarithm of a multiplication of two numbers is the sum of logarithms of the two numbers taken separately. The results is abbreviated as log-probability.

15. Write class ```MarkovLog``` that given the $k$-th order Markov chain developed above to the constructor, its method ```log_probability``` computes the log-probability of a given input DNA sequence. Run your program with $T=$ `ATGATATCATCGACGATGTAG` and $k=2$.

In [ ]:
class MarkovLog(object):
    def __init__(self, k: int, zeroth: Dict[str, float], kth: Dict[str, Dict[str, float]]):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
        
    def log_probability(self, seq: str) -> float:
        if not seq:
            return np.nan
        
        probability = 0
        for i, s in enumerate(seq):
            if i < self.k:
                probability += np.log2(self.zeroth[s])
                continue

            context = seq[i-self.k:i]
            probability += np.log2(self.kth[context][s])
        
        return probability
    
if __name__ == '__main__':
    k = 2
    kth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", k)
    zeroth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", 0)[""]
    mc = MarkovLog(2, zeroth, kth)
    s="ATGATATCATCGACGATGTAG"
    print(f"Log probability of sequence {s} is {mc.log_probability(s)}")

Log probability of sequence ATGATATCATCGACGATGTAG is -31.717831515538315


### Idea of solution

The only difference to the previous exercise is the use of $\log_2$ and summation instead of multiplication for the probabilities.

### Discussion

The user is responsible for providing valid input.

Finally, if you try to use the code so far for very large inputs, you might observe that the concatenation of symbols following a context occupy considerable amount of space. This is unnecessary, as we only need the frequencies. 

16. Optimize the space requirement of your code from exercise 13 for the $k$-th order Markov chain by replacing the concatenations by direct computations of the frequencies. Implement this as the
  ```better_context_probabilities``` function.

In [ ]:
def better_context_probabilities(seq: str, k: int):
    probabilities = {}
    for kmer in product("ACGT", repeat=k):
        probabilities["".join(kmer)] = Counter("ACGT")

    for i in range(len(seq)-k):
        context = seq[i:i+k]
        symbol = seq[i+k]
        probabilities[context].update(symbol)
    
    for context, counts in probabilities.items():
        total = sum(counts.values())
        probabilities[context] = {sym: cnt/total for sym, cnt in counts.items()}

    return probabilities

if __name__ == '__main__':
    k = 1
    s = "ATGATATCATCGACGATGTAG"
    d = better_context_probabilities(s, k)
    print("\n".join(f"{k}: {v}" for k, v in d.items()))

A: {'A': 0.09090909090909091, 'C': 0.18181818181818182, 'G': 0.18181818181818182, 'T': 0.5454545454545454}
C: {'A': 0.2857142857142857, 'C': 0.14285714285714285, 'G': 0.42857142857142855, 'T': 0.14285714285714285}
G: {'A': 0.5, 'C': 0.125, 'G': 0.125, 'T': 0.25}
T: {'A': 0.3, 'C': 0.3, 'G': 0.3, 'T': 0.1}


### Idea of solution

The input string is iterated over in $k+1$ -sized chunks, with the first $k$ characters being the context, and the last the symbol. The counts are directly updated to a counter and finally their proportions are calculated for each context.

### Discussion

With the strings associated to the contexts not stored, the major memory use is the space required to store the $4^k$ contexts.

While the earlier approach of explicit concatenation of symbols following a context suffered from inefficient use of space, it does have a benefit of giving another much simpler strategy to sample from the distribution: 
observe that an element of the concatenation taken uniformly randomly is sampled exactly with the correct probability. 

17. Revisit the solution 12 and modify it to directly sample from the concatenation of symbols following a context. The function ```np.random.choice``` may be convenient here. Implement the modified version as the new `SimpleMarkovChain` class.

In [ ]:
class SimpleMarkovChain(object):
    def __init__(self, dna_seq: str, k: int):
        self.k = k
        self.zeroth = context_list(dna_seq, 0)[""]
        self.kth = context_list(dna_seq, k, pseudo_fill="ACGT")

    def generate(self, n: int, seed=None) -> str:
        rng = random.RandomState(seed)

        seq = [rng.choice(list(self.zeroth)) for _ in range(min(n, self.k))]
        for i in range(self.k, n):
            context = "".join(seq[i-self.k:])
            seq.append(rng.choice(list(self.kth[context])))

        return "".join(seq)
        
if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    n = 40
    seed = 0
    mc = SimpleMarkovChain(s, k)
    print(mc.generate(n, seed))

AACATTTCTCGATGATCACCCACGTATAAGTACTCTTGAC


### Idea of solution

The first $\min(n, k)$ symbols are chosen by selecting a random key from the zeroth distribution, and the rest using the $k$ latest symbols as a context.

### Discussion

String building from a list of characters is used due to its speed advantage over string concatenation.

## $k$-mer index

Our $k$-th order Markov chain can now be modified to a handy index structure called $k$-mer index. This index structure associates to each $k$-mer its list of occurrence positions in DNA sequence $T$.  Given a query $k$-mer $W$, one can thus easily list all positions $i$ with  $T[i..k-1]=W$.

18. Implement function ```kmer_index``` inspired by your earlier code for the $k$-th order Markov chain. Test your program with `ATGATATCATCGACGATGTAG` and $k=2$.

In [ ]:
def kmer_index(seq: str, k: int) -> Dict[str, List[int]]:
    indices = defaultdict(list)

    for i in range(len(seq) - k + 1):
        context = seq[i:i+k] # k-length slice starting at i
        indices[context].append(i)
    
    return indices

if __name__ == '__main__':
    k=2
    s = "ATGATATCATCGACGATGTAG"
    print("Using string:")
    print(s)
    print("".join([str(i%10) for i in range(len(s))]))
    print(f"\n{k}-mer index is:")
    d=kmer_index(s, k)
    print(dict(d))

Using string:
ATGATATCATCGACGATGTAG
012345678901234567890

2-mer index is:
{'AT': [0, 3, 5, 8, 15], 'TG': [1, 16], 'GA': [2, 11, 14], 'TA': [4, 18], 'TC': [6, 9], 'CA': [7], 'CG': [10, 13], 'AC': [12], 'GT': [17], 'AG': [19]}


### Idea of solution

The function goes through all the $k$-mers in the string in a for-loop and stores their starting indices in a defaultdict as a list.

### Discussion

The context slicing increases the time complexity of the function, but I was unable to skip the slicing step while retaining a sufficient ability to generate required substrings for the keys.

## Comparison of probability distributions

Now that we know how to learn probability distributions from data, we might want to compare two such distributions, for example, to test if our programs work as intended. 

Let $P=\{p_1,p_2,\ldots, p_n\}$ and $Q=\{q_1,q_2,\ldots, q_n\}$ be two probability distributions for the same set of $n$ events. This means $\sum_{i=1}^n p_i=\sum_{i=1}^n q_i=1$, $0\leq p_j \leq 1$, and $0\leq q_j \leq 1$ for each event $j$. 

*Kullback-Leibler divergence* is a measure $d()$ for the *relative entropy* of $P$ with respect to $Q$ defined as 
$d(P||Q)=\sum_{i=1}^n p_i \log\frac{p_i}{q_i}$.


This measure is always non-negative, and 0 only when $P=Q$. It can be interpreted as the gain of knowing $Q$ to encode $P$. Note that this measure is not symmetric.

19. Write function ```kullback_leibler``` to compute $d(P||Q)$. Test your solution by generating a random RNA sequence
  encoding the input protein sequence according to the input codon adaptation probabilities.
  Then you should learn the codon adaptation probabilities from the RNA sequence you generated.
  Then try the same with uniformly random RNA sequences (which don't have to encode any
  specific protein sequence). Compute the relative entropies between the
  three distribution (original, predicted, uniform) and you should observe a clear difference.
  Because $d(P||Q)$ is not symmetric, you can either print both $d(P||Q)$ and $d(Q||P)$,
  or their average.
  
  This problem may be fairly tricky. Only the `kullback_leibler` function is automatically tested. The codon probabilities is probably a useful helper function. The main guarded section can be completed by filling out the `pass` sections using tooling from previous parts and fixing the *placeholder* lines.

In [ ]:
def codon_probabilities(seq: str, *, k: int = 3) -> Dict[str, float]:
    """
    Given an RNA sequence, simply calculates the probability of
    all 3-mers empirically based on the sequence
    """
    kmers = kmer_index(seq, k)
    total = sum(map(len, kmers.values())) # k-mer population size
    return {k: len(v)/total for k, v in kmers.items()} # probabilities of each k-mer

def ungrouped_codon_probabilities() -> Dict[str, float]:
    data = read_codon_table()
    # Calculate the probability of each 3-mer over the whole population, not grouped by protein.
    data["Probability"] = data["Number"] / data["Number"].sum()

    return dict(zip(data["Triplet"], data["Probability"]))
    
def kullback_leibler(p: Dict[Any, float], q: Dict[Any, float]):
    """
    Computes Kullback-Leibler divergence between two distributions.
    Both p and q must be dictionaries from events to probabilities.
    The divergence is defined only when q[event] == 0 implies p[event] == 0.
    """
    # Find the events shared between both distributions.
    common_evts = set(p).intersection(set(q))

    # Sum those events together for which log is defined, i.e. p > 0
    result = sum(p[evt] * np.log2(p[evt]/q[evt]) for evt in common_evts if p[evt] > 0)

    return result

if __name__ == '__main__':
    aas = list("*ACDEFGHIKLMNPQRSTVWY") # List of amino acids
    n = 10000
    
    # generate a random protein and some associated rna
    protein = "".join(choice(aas, n))
    rna = ProteinToRandomRNA().convert(protein)
    
    # Maybe check that converting back to protein results in the same sequence
    assert rna_to_prot(rna) == protein
    
    # Calculate codon probabilities of the rna sequence
    cp_predicted = codon_probabilities(rna)
    
    # Calculate codon probabilities based on the codon usage table
    cp_orig = ungrouped_codon_probabilities()
    
    # Create a completely random RNA sequence and get the codon probabilities
    uniform_rna = "".join(choice(list("ACGU"), len(rna)))
    cp_uniform = codon_probabilities(uniform_rna)

    print("d(original || predicted) =", kullback_leibler(cp_orig, cp_predicted))
    print("d(predicted || original) =", kullback_leibler(cp_predicted, cp_orig))
    print()
    print("d(original || uniform) =", kullback_leibler(cp_orig, cp_uniform))
    print("d(uniform || original) =", kullback_leibler(cp_uniform, cp_orig))
    print()
    print("d(predicted || uniform) =", kullback_leibler(cp_predicted, cp_uniform))
    print("d(uniform || predicted) =", kullback_leibler(cp_uniform, cp_predicted))

d(original || predicted) = 0.16363814225143006
d(predicted || original) = 0.22070150670270408

d(original || uniform) = 0.20170163877990724
d(uniform || original) = 0.267695966490279

d(predicted || uniform) = 0.06237557512778992
d(uniform || predicted) = 0.06499383794817162


### Idea of solution

The KL function first finds the events shared by both distributions, and then iterates over those for which $\log_2$ is defined, i.e. when $p_i > 0$.

I added an implementation for codon_probabilities that calculates the probabilities over the whole population, rather than for each protein.

### Discussion

The predicted and uniform distributions are more similar to each other ($\approx 0.06$) than to the original distribution ($\approx 0.16$ and $\approx 0.20$, respectively).

## Stationary and equilibrium distributions (extra)

Let us consider a Markov chain of order one on the set of nucleotides.
Its transition probabilities can be expressed as a $4 \times 4$ matrix
$P=(p_{ij})$, where the element $p_{ij}$ gives the probability of the $j$th nucleotide
on the condition the previous nucleotide was the $i$th. An example of a transition matrix
is

\begin{array}{l|rrrr}
 &     A &    C &     G &    T \\
\hline
A &  0.30 &  0.0 &  0.70 &  0.0 \\
C &  0.00 &  0.4 &  0.00 &  0.6 \\
G &  0.35 &  0.0 &  0.65 &  0.0 \\
T &  0.00 &  0.2 &  0.00 &  0.8 \\
\end{array}.

A distribution $\pi=(\pi_1,\pi_2,\pi_3,\pi_4)$ is called *stationary*, if
$\pi = \pi P$ (the product here is matrix product).

20. Write function ```get_stationary_distributions``` that gets a transition matrix as parameter,
  and returns the list of stationary distributions. You can do this with NumPy by
  first taking transposition of both sides of the above equation to get equation
  $\pi^T = P^T \pi^T$. Using numpy.linalg.eig take all eigenvectors related to
  eigenvalue 1.0. By normalizing these vectors to sum up to one get the stationary distributions
  of the original transition matrix. In the ```main``` function print the stationary distributions
  of the above transition matrix.

In [ ]:
def get_stationary_distributions(transition: np.ndarray) -> np.ndarray:
    """
    The function get a transition matrix of a degree one Markov chain as parameter.
    It returns a list of stationary distributions, in vector form, for that chain.
    """
    eig_vals, eig_vecs = np.linalg.eig(transition.T)
    eig_vecs = eig_vecs[:, np.isclose(eig_vals, 1)]
    eig_vecs = eig_vecs / np.sum(eig_vecs, axis=0)
    return eig_vecs.T
    
    
if __name__ == "__main__":
    transition=np.array([[0.3, 0, 0.7, 0],
                         [0, 0.4, 0, 0.6],
                         [0.35, 0, 0.65, 0],
                         [0, 0.2, 0, 0.8]])
    
    print("\n".join(
        ", ".join(
            f"{pv:+.3f}"
            for pv in p) 
        for p in get_stationary_distributions(transition)))

+0.333, +0.000, +0.667, +0.000
-0.000, +0.250, -0.000, +0.750


### Idea of solution


### Discussion


21. Implement the `kl_divergence` function below so that the main guarded code runs properly. Using your modified Markov chain generator generate a nucleotide sequence $s$ of length $10\;000$. Choose prefixes of $s$ of lengths $1, 10, 100, 1000$, and $10\;000$. For each of these prefixes find out their nucleotide distribution (of order 0) using your earlier tool. Use 1 as the pseudo count. Then, for each prefix, compute the KL divergence between the initial distribution and the normalized nucleotide distribution.

In [ ]:
def kl_divergences(initial, transition):
    """
    Calculates the the Kullback-Leibler divergences between empirical distributions
    generated using a markov model seeded with an initial distributin and a transition 
    matrix, and the initial distribution.
    Sequences of length [1, 10, 100, 1000, 10000] are generated.
    """
    base = dict(zip("ACGT", initial @ transition))
    s = MarkovChain(base, base, 0).generate(10000)

    lengths = [1, 10, 100, 1000, 10000]
    results = []

    for length in lengths:
        ss = s[:length]
        dist = codon_probabilities(ss, k=1)
        results.append(kullback_leibler(dist, dict(zip("ACGT", initial))))

    return zip(lengths, results)

if __name__ == "__main__":
    transition=np.array([[0.3, 0, 0.7, 0],
                         [0, 0.4, 0, 0.6],
                         [0.35, 0, 0.65, 0],
                         [0, 0.2, 0, 0.8]])
    print("Transition probabilities are:")
    print(transition)
    stationary_distributions = get_stationary_distributions(transition)
    print("Stationary distributions:")
    print(np.stack(stationary_distributions))
    initial = stationary_distributions[1]
    print("Using [{}] as initial distribution\n".format(", ".join(f"{v:.2f}" for v in initial)))
    results = kl_divergences(initial, transition)
    for prefix_length, divergence in results: # iterate on prefix lengths in order (1, 10, 100...)
        print("KL divergence of stationary distribution prefix " \
              "of length {:5d} is {:.8f}".format(prefix_length, divergence))

Transition probabilities are:
[[0.3  0.   0.7  0.  ]
 [0.   0.4  0.   0.6 ]
 [0.35 0.   0.65 0.  ]
 [0.   0.2  0.   0.8 ]]
Stationary distributions:
[[ 0.33333333  0.          0.66666667  0.        ]
 [-0.          0.25       -0.          0.75      ]]
Using [-0.00, 0.25, -0.00, 0.75] as initial distribution

KL divergence of stationary distribution prefix of length     1 is 0.41503750
KL divergence of stationary distribution prefix of length    10 is 0.00923535
KL divergence of stationary distribution prefix of length   100 is -0.00000000
KL divergence of stationary distribution prefix of length  1000 is 0.00006134
KL divergence of stationary distribution prefix of length 10000 is 0.00006812


### Idea of solution

fill in

### Discussion
fill in

22. Implement the following in the ```main``` function.
Find the stationary distribution for the following transition matrix:  

\begin{array}{ l | r r r r}
 & A &     C &     G &     T \\
\hline
A &  0.30 &  0.10 &  0.50 &  0.10 \\
C &  0.20 &  0.30 &  0.15 &  0.35 \\
G &  0.25 &  0.15 &  0.20 &  0.40 \\
T &  0.35 &  0.20 &  0.40 &  0.05 \\
\end{array}

Since there is only one stationary distribution, it is called the *equilibrium distribution*.
Choose randomly two nucleotide distributions. You can take these from your sleeve or
sample them from the Dirichlet distribution. Then for each of these distributions
as the initial distribution of the Markov chain, repeat the above experiment.

The `main` function should return tuples, where the first element is the (random) initial distribution and the second element contains the results as a list of tuples where the first element is the kl divergence and the second element the empirical nucleotide distribution, for the different prefix lengths.

The state distribution should converge to the equilibrium distribution no matter how we
start the Markov chain! That is the last line of the tables should have KL-divergence very close to $0$ and an empirical distribution very close to the equilibrium distribution.


In [ ]:
def main(transition, equilibrium_distribution):
    vals = list(zip(np.random.rand(10), np.random.rand(10, 4) - 0.5))
    return zip(np.random.rand(2, 4) - 0.5, 
               [vals[:5], vals[5:]])


if __name__ == "__main__":
    transition = np.array([[0.3, 0.1, 0.5, 0.1],
                           [0.2, 0.3, 0.15, 0.35],
                           [0.25, 0.15, 0.2, 0.4],
                           [0.35, 0.2, 0.4, 0.05]])
    print("Transition probabilities are:", transition, sep="\n")
    stationary_distributions = get_stationary_distributions(transition)
    # Uncomment the below line to check that there actually is only one stationary distribution
    # assert len(stationary_distributions) == 1
    equilibrium_distribution = stationary_distributions[0]
    print("Equilibrium distribution:")
    print(equilibrium_distribution)
    for initial_distribution, results in main(transition, equilibrium_distribution):
        print("\nUsing {} as initial distribution:".format(initial_distribution))
        print("kl-divergence   empirical distribution")
        print("\n".join("{:.11f}   {}".format(di, kl) for di, kl in results))

Transition probabilities are:
[[0.3  0.1  0.5  0.1 ]
 [0.2  0.3  0.15 0.35]
 [0.25 0.15 0.2  0.4 ]
 [0.35 0.2  0.4  0.05]]
Equilibrium distribution:
[0.27803345 0.17353238 0.32035021 0.22808396]

Using [ 0.17975591 -0.20922371  0.32117689 -0.27225399] as initial distribution:
kl-divergence   empirical distribution
0.58591719300   [ 0.29088841 -0.4205835   0.17791163 -0.20722116]
0.04859529815   [ 0.15446201 -0.46100207  0.29696647  0.18748396]
0.86887609462   [ 0.15055897  0.08888351  0.1497692  -0.21313654]
0.28444251533   [-0.41637382 -0.08710792  0.22572366 -0.48754792]
0.33054899251   [ 0.09675373 -0.03431931  0.15320762 -0.13971118]

Using [ 0.06593716 -0.25911655  0.07238256 -0.09408949] as initial distribution:
kl-divergence   empirical distribution
0.74700447978   [-0.1646051   0.17620919 -0.3929642   0.15765592]
0.18372119013   [-0.02364586  0.25804057 -0.49913742  0.18561779]
0.95640347847   [ 0.02574726  0.37353413  0.46898165 -0.38449805]
0.48353560775   [ 0.05299729 -0.369

### Idea of solution

fill in

### Discussion
fill in